In [22]:
file = spark.sparkContext.textFile('hdfs://localhost:9000/XYonTimeArrPerf/part-r-00000')

In [23]:
file.take(10)

['ABE ATL DL\t4.804878048780488',
 'ABE CLT US\t-0.30337078651685395',
 'ABE CVG DL\t-6.345238095238095',
 'ABQ AMA WN\t7.504201680672269',
 'ABQ ATL DL\t1.729281767955801',
 'ABQ COS DL\t3.164705882352941',
 'ABQ CVG DL\t6.670454545454546',
 'ABQ DAL WN\t7.606260296540363',
 'ABQ DEN UA\t5.16722972972973',
 'ABQ DFW AA\t4.222648752399232']

In [24]:
file.map(lambda line:line.split()).take(10)

[['ABE', 'ATL', 'DL', '4.804878048780488'],
 ['ABE', 'CLT', 'US', '-0.30337078651685395'],
 ['ABE', 'CVG', 'DL', '-6.345238095238095'],
 ['ABQ', 'AMA', 'WN', '7.504201680672269'],
 ['ABQ', 'ATL', 'DL', '1.729281767955801'],
 ['ABQ', 'COS', 'DL', '3.164705882352941'],
 ['ABQ', 'CVG', 'DL', '6.670454545454546'],
 ['ABQ', 'DAL', 'WN', '7.606260296540363'],
 ['ABQ', 'DEN', 'UA', '5.16722972972973'],
 ['ABQ', 'DFW', 'AA', '4.222648752399232']]

In [26]:
file.map(lambda line:line.split()).filter(lambda tuple:tuple[0]=='ABQ').filter(lambda tuple:tuple[1]=='DFW').take(5)

[['ABQ', 'DFW', 'AA', '4.222648752399232'],
 ['ABQ', 'DFW', 'DL', '5.601449275362318']]

In [27]:
file = file.map(lambda line:line.split()).filter(lambda tuple:tuple[0]=='ABQ').filter(lambda tuple:tuple[1]=='DFW') #.map(lambda tuple:(float(tuple[3]),tuple[2])).sortByKey(ascending=False).take(10)

In [28]:
file1 = file.map(lambda tuple:(tuple[0],tuple[1],float(tuple[3])))

In [29]:
file1.take(5)


[('ABQ', 'DFW', 4.222648752399232), ('ABQ', 'DFW', 5.601449275362318)]

In [13]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])  # provide contact points and port
session = cluster.connect('aviation')

In [10]:
#query = "INSERT INTO airport_srcdest_arrival (src, dest, arr_delay) VALUES (?, ?, ?)"
#prepared = session.prepare(query)

In [ ]:
#session.execute(prepared, (row.select('src'), row.select('dest'),row.select('arr_delay')))

In [58]:
from pyspark.sql import Row
rdd3 = file1.map(lambda row: Row(src=row[0], dest=row[1], arr_delay=row[2]))


In [59]:
df = spark.createDataFrame(rdd3)

In [60]:
df.show()

+-----------------+----+---+
|        arr_delay|dest|src|
+-----------------+----+---+
|4.222648752399232| DFW|ABQ|
|5.601449275362318| DFW|ABQ|
+-----------------+----+---+



In [34]:
from pyspark.sql.functions import round, col
df = df.withColumn("arr_delay", format_number(df.arr_delay, 2))

NameError: name 'format_number' is not defined

In [29]:
df.show()


AttributeError: 'NoneType' object has no attribute 'show'

In [22]:
pip install cassandra-driver


Note: you may need to restart the kernel to use updated packages.


In [42]:
from cassandra.cluster import Cluster

In [43]:
cluster = Cluster(['127.0.0.1'])  # provide contact points and port
session = cluster.connect('aviation')

In [40]:
rows = session.execute('select * from airport_srcdest_arrival;')

In [41]:
for row in rows:
    print(row)

Row(src='APT', dest='SDF', arr_delay=Decimal('-9.8'))


In [52]:
query = "INSERT INTO airport_srcdest_arrival (src, dest, arr_delay) VALUES (?, ?, ?)"


In [30]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(spark)

In [54]:
prepared = session.prepare(query)

In [55]:
session.execute(prepared, ("DAD", "SDA", -2.34))

In [57]:
rows = session.execute('select * from airport_srcdest_arrival;')
for row in rows:
    print(row)

Row(src='APT', dest='SDF', arr_delay=Decimal('-9.8'))
Row(src='DAD', dest='SDA', arr_delay=Decimal('-2.339999999999999857891452847979962825775146484375'))


In [16]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(spark)

In [62]:
ds = sqlContext.read \
  .format('org.apache.spark.sql.cassandra') \
  .options(table='airport_srcdest_arrival', keyspace='aviation') \
  .load()

In [63]:
ds.show()

+---+----+--------------------+
|src|dest|           arr_delay|
+---+----+--------------------+
|ABQ| DFW|4.222648752399232000|
|ABQ| DFW|5.601449275362318000|
+---+----+--------------------+



In [1]:
df.write\
.format("org.apache.spark.sql.cassandra")\
.mode('append')\
.options(table = "airport_srcdest_arrival", keyspace = "aviation")  \
.save()

NameError: name 'df' is not defined